## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Лабораторная работа 3

In [1]:
# !pip install line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [3]:
import pandas as pd
recipes = pd.read_csv('recipes_sample.csv', parse_dates=['submitted'])
reviews = pd.read_csv('reviews_sample.csv',index_col = 0,  parse_dates=['date'])
recipes

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16.0,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,NaN,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,NaN,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,227978,2012-08-29,NaN,this is a delicious soup that i originally fou...,NaN


In [4]:
%%time
#A. С использованием метода DataFrame.iterrows исходной таблицы;
def count_rating1():
    reviews = pd.read_csv('reviews_sample.csv',index_col = 0,  parse_dates=['date'])
    summ = 0
    count = 0
    for index, row in reviews.iterrows():
        if row['date'].year==2010:
            count+=1
            summ+=row['rating']
    answer1 = summ/count
    return answer1
count_rating1()

Wall time: 2.99 s


4.4544402182900615

In [5]:
%%time
#Б. С использованием метода DataFrame.iterrows таблицы, в которой сохранены только отзывы за 2010 год;
def count_rating2():
    reviews = pd.read_csv('reviews_sample.csv',index_col = 0,  parse_dates=['date'])
    summ = 0 
    reviews1 = reviews[reviews['date'].dt.year==2010]
    for index, row in reviews1.iterrows():
        summ+=row['rating']

    answer2 = (summ/reviews1.shape[0])
    return answer2
count_rating2()

Wall time: 660 ms


4.4544402182900615

In [6]:
%%time
#В. С использованием метода Series.mean
def count_rating3():
    reviews = pd.read_csv('reviews_sample.csv',index_col = 0,  parse_dates=['date'])
    answer3 = reviews[reviews['date'].dt.year==2010].rating.mean()
    return answer3
count_rating3()

Wall time: 434 ms


4.4544402182900615

In [7]:
count_rating1()==count_rating2()==count_rating3()

True

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [8]:
!pip install line_profiler

     ---------------------------------------- 83.6/83.6 kB 1.6 MB/s eta 0:00:00


In [9]:
!pip install memory_profiler

In [10]:
%load_ext line_profiler

In [11]:
%reload_ext line_profiler

In [12]:
%load_ext memory_profiler

In [13]:
%%writefile demo.py
def count_rating1():
    reviews = pd.read_csv('reviews_sample.csv',index_col = 0,  parse_dates=['date'])
    summ = 0
    count = 0
    for index, row in reviews.iterrows():
        if row['date'].year==2010:
            count+=1
            summ+=row['rating']
    answer1 = summ/count
    return answer1

Writing demo.py


In [14]:
import cProfile, pstats, io


def profile(fnc):
    
    """A decorator that uses cProfile to profile a function"""
    
    def inner(*args, **kwargs):
        
        pr = cProfile.Profile()
        pr.enable()
        retval = fnc(*args, **kwargs)
        pr.disable()
        s = io.StringIO()
        sortby = 'cumulative'
        ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
        ps.print_stats()
        print(s.getvalue())
        return retval

    return inner
@profile
def count_rating1():
    reviews = pd.read_csv('reviews_sample.csv',index_col = 0,  parse_dates=['date'])
    summ = 0
    count = 0
    for index, row in reviews.iterrows():
        if row['date'].year==2010:
            count+=1
            summ+=row['rating']
    answer1 = summ/count
    return answer1

In [15]:
count_rating1()

         16361089 function calls (16107660 primitive calls) in 5.791 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.115    0.115    5.791    5.791 C:\Users\hapyl\AppData\Local\Temp\ipykernel_14200\3444025220.py:22(count_rating1)
   126697    0.108    0.000    4.295    0.000 C:\Users\hapyl\anaconda3\lib\site-packages\pandas\core\frame.py:1279(iterrows)
126702/126699    0.531    0.000    4.108    0.000 C:\Users\hapyl\anaconda3\lib\site-packages\pandas\core\series.py:323(__init__)
   126708    0.294    0.000    1.185    0.000 C:\Users\hapyl\anaconda3\lib\site-packages\pandas\core\construction.py:470(sanitize_array)
   138790    0.180    0.000    0.948    0.000 C:\Users\hapyl\anaconda3\lib\site-packages\pandas\core\series.py:943(__getitem__)
   126699    0.104    0.000    0.796    0.000 C:\Users\hapyl\anaconda3\lib\site-packages\pandas\core\internals\managers.py:1745(from_array)
   138790    0.108    0.000    

4.4544402182900615

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [16]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count